In [16]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")

# db = client["Monitoring"]
# collection = db["PRJ-16"]

db = client["SteelArena"]
collection = db["Init"]

In [3]:
combined_pipeline = [
    {
        '$match': {
            '$or': [
                {'T1.t': {'$exists': True}, 'A30.t': {'$exists': True}},
                {'T2.t': {'$exists': True}, 'A31.t': {'$exists': True}},
                {'T3.t': {'$exists': True}, 'A32.t': {'$exists': True}}
            ]
        }
    },
    {
        '$addFields': {
            'A30_filtered': {
                '$cond': {
                    'if': {'$gt': ['$A30.t', 60]},
                    'then': None,
                    'else': '$A30.t'
                }
            },
            'A31_filtered': {
                '$cond': {
                    'if': {'$gt': ['$A31.t', 60]},
                    'then': None,
                    'else': '$A31.t'
                }
            },
            'A32_filtered': {
                '$cond': {
                    'if': {'$gt': ['$A32.t', 60]},
                    'then': None,
                    'else': '$A32.t'
                }
            }
        }
    },
    {
        '$addFields': {
            'hour': {
                '$dateTrunc': {
                    'date': {'$toDate': '$time.datetime'},
                    'unit': 'hour'
                }
            }
        }
    },
    {
        '$group': {
            '_id': '$hour',
            'T1_values': {'$push': '$T1.t'},
            'A30_values': {'$push': '$A30_filtered'},
            'T2_values': {'$push': '$T2.t'},
            'A31_values': {'$push': '$A31_filtered'},
            'T3_values': {'$push': '$T3.t'},
            'A32_values': {'$push': '$A32_filtered'}
        }
    },
    {
        '$project': {
            'hour': '$_id',
            '_id': 0,
            'median_T1': {
                '$let': {
                    'vars': {
                        'sorted_T1': {
                            '$sortArray': {
                                'input': '$T1_values',
                                'sortBy': 1
                            }
                        }
                    },
                    'in': {
                        '$arrayElemAt': [
                            '$$sorted_T1', {
                                '$floor': {
                                    '$divide': [
                                        {'$size': '$$sorted_T1'}, 2
                                    ]
                                }
                            }
                        ]
                    }
                }
            },
            'median_A30': {
                '$cond': {
                    'if': {'$gt': [{'$size': {'$filter': {'input': '$A30_values', 'as': 'value', 'cond': {'$ne': ['$$value', None]}}}}, 0]},
                    'then': {
                        '$let': {
                            'vars': {
                                'sorted_A30': {
                                    '$sortArray': {
                                        'input': {'$filter': {'input': '$A30_values', 'as': 'value', 'cond': {'$ne': ['$$value', None]}}},
                                        'sortBy': 1
                                    }
                                }
                            },
                            'in': {
                                '$arrayElemAt': [
                                    '$$sorted_A30', {
                                        '$floor': {
                                            '$divide': [
                                                {'$size': '$$sorted_A30'}, 2
                                            ]
                                        }
                                    }
                                ]
                            }
                        }
                    },
                    'else': None
                }
            },
            'median_T2': {
                '$let': {
                    'vars': {
                        'sorted_T2': {
                            '$sortArray': {
                                'input': '$T2_values',
                                'sortBy': 1
                            }
                        }
                    },
                    'in': {
                        '$arrayElemAt': [
                            '$$sorted_T2', {
                                '$floor': {
                                    '$divide': [
                                        {'$size': '$$sorted_T2'}, 2
                                    ]
                                }
                            }
                        ]
                    }
                }
            },
            'median_A31': {
                '$cond': {
                    'if': {'$gt': [{'$size': {'$filter': {'input': '$A31_values', 'as': 'value', 'cond': {'$ne': ['$$value', None]}}}}, 0]},
                    'then': {
                        '$let': {
                            'vars': {
                                'sorted_A31': {
                                    '$sortArray': {
                                        'input': {'$filter': {'input': '$A31_values', 'as': 'value', 'cond': {'$ne': ['$$value', None]}}},
                                        'sortBy': 1
                                    }
                                }
                            },
                            'in': {
                                '$arrayElemAt': [
                                    '$$sorted_A31', {
                                        '$floor': {
                                            '$divide': [
                                                {'$size': '$$sorted_A31'}, 2
                                            ]
                                        }
                                    }
                                ]
                            }
                        }
                    },
                    'else': None
                }
            },
            'median_T3': {
                '$let': {
                    'vars': {
                        'sorted_T3': {
                            '$sortArray': {
                                'input': '$T3_values',
                                'sortBy': 1
                            }
                        }
                    },
                    'in': {
                        '$arrayElemAt': [
                            '$$sorted_T3', {
                                '$floor': {
                                    '$divide': [
                                        {'$size': '$$sorted_T3'}, 2
                                    ]
                                }
                            }
                        ]
                    }
                }
            },
            'median_A32': {
                '$cond': {
                    'if': {'$gt': [{'$size': {'$filter': {'input': '$A32_values', 'as': 'value', 'cond': {'$ne': ['$$value', None]}}}}, 0]},
                    'then': {
                        '$let': {
                            'vars': {
                                'sorted_A32': {
                                    '$sortArray': {
                                        'input': {'$filter': {'input': '$A32_values', 'as': 'value', 'cond': {'$ne': ['$$value', None]}}},
                                        'sortBy': 1
                                    }
                                }
                            },
                            'in': {
                                '$arrayElemAt': [
                                    '$$sorted_A32', {
                                        '$floor': {
                                            '$divide': [
                                                {'$size': '$$sorted_A32'}, 2
                                            ]
                                        }
                                    }
                                ]
                            }
                        }
                    },
                    'else': None
                }
            }
        }
    },
    {
        '$sort': {
            'hour': 1
        }
    }
]

In [17]:
combined_pipeline = [
    {
        '$match': {
            '$or': [
                {
                    'T1.t': {
                        '$exists': True
                    },
                    'A30.t': {
                        '$exists': True
                    }
                }, {
                    'T2.t': {
                        '$exists': True
                    },
                    'A31.t': {
                        '$exists': True
                    }
                }, {
                    'T3.t': {
                        '$exists': True
                    },
                    'A32.t': {
                        '$exists': True
                    }
                }, {
                    'T4.t': {
                        '$exists': True
                    },
                    'A33.t': {
                        '$exists': True
                    }
                }, {
                    'T5.t': {
                        '$exists': True
                    },
                    'A34.t': {
                        '$exists': True
                    }
                }, {
                    'T6.t': {
                        '$exists': True
                    },
                    'A35.t': {
                        '$exists': True
                    }
                }
            ]
        }
    }, {
        '$addFields': {
            'A30_filtered': {
                '$cond': {
                    'if': {
                        '$gt': [
                            '$A30.t', 60
                        ]
                    },
                    'then': None,
                    'else': '$A30.t'
                }
            },
            'A31_filtered': {
                '$cond': {
                    'if': {
                        '$gt': [
                            '$A31.t', 60
                        ]
                    },
                    'then': None,
                    'else': '$A31.t'
                }
            },
            'A32_filtered': {
                '$cond': {
                    'if': {
                        '$gt': [
                            '$A32.t', 60
                        ]
                    },
                    'then': None,
                    'else': '$A32.t'
                }
            },
            'A33_filtered': {
                '$cond': {
                    'if': {
                        '$gt': [
                            '$A33.t', 60
                        ]
                    },
                    'then': None,
                    'else': '$A33.t'
                }
            },
            'A34_filtered': {
                '$cond': {
                    'if': {
                        '$gt': [
                            '$A34.t', 60
                        ]
                    },
                    'then': None,
                    'else': '$A34.t'
                }
            },
            'A35_filtered': {
                '$cond': {
                    'if': {
                        '$gt': [
                            '$A35.t', 60
                        ]
                    },
                    'then': None,
                    'else': '$A35.t'
                }
            }
        }
    }, {
        '$addFields': {
            'hour': {
                '$dateTrunc': {
                    'date': {
                        '$toDate': '$time.datetime'
                    },
                    'unit': 'hour'
                }
            }
        }
    }, {
        '$group': {
            '_id': '$hour',
            'T1_values': {
                '$push': '$T1.t'
            },
            'A30_values': {
                '$push': '$A30_filtered'
            },
            'T2_values': {
                '$push': '$T2.t'
            },
            'A31_values': {
                '$push': '$A31_filtered'
            },
            'T3_values': {
                '$push': '$T3.t'
            },
            'A32_values': {
                '$push': '$A32_filtered'
            },
            'T4_values': {
                '$push': '$T4.t'
            },
            'A33_values': {
                '$push': '$A33_filtered'
            },
            'T5_values': {
                '$push': '$T5.t'
            },
            'A34_values': {
                '$push': '$A34_filtered'
            },
            'T6_values': {
                '$push': '$T6.t'
            },
            'A35_values': {
                '$push': '$A35_filtered'
            }
        }
    }, {
        '$project': {
            'hour': '$_id',
            '_id': 0,
            'median_T1': {
                '$let': {
                    'vars': {
                        'sorted_T1': {
                            '$sortArray': {
                                'input': '$T1_values',
                                'sortBy': 1
                            }
                        }
                    },
                    'in': {
                        '$arrayElemAt': [
                            '$$sorted_T1', {
                                '$floor': {
                                    '$divide': [
                                        {
                                            '$size': '$$sorted_T1'
                                        }, 2
                                    ]
                                }
                            }
                        ]
                    }
                }
            },
            'median_A30': {
                '$cond': {
                    'if': {
                        '$gt': [
                            {
                                '$size': {
                                    '$filter': {
                                        'input': '$A30_values',
                                        'as': 'value',
                                        'cond': {
                                            '$ne': [
                                                '$$value', None
                                            ]
                                        }
                                    }
                                }
                            }, 0
                        ]
                    },
                    'then': {
                        '$let': {
                            'vars': {
                                'sorted_A30': {
                                    '$sortArray': {
                                        'input': {
                                            '$filter': {
                                                'input': '$A30_values',
                                                'as': 'value',
                                                'cond': {
                                                    '$ne': [
                                                        '$$value', None
                                                    ]
                                                }
                                            }
                                        },
                                        'sortBy': 1
                                    }
                                }
                            },
                            'in': {
                                '$arrayElemAt': [
                                    '$$sorted_A30', {
                                        '$floor': {
                                            '$divide': [
                                                {
                                                    '$size': '$$sorted_A30'
                                                }, 2
                                            ]
                                        }
                                    }
                                ]
                            }
                        }
                    },
                    'else': None
                }
            },
            'median_T2': {
                '$let': {
                    'vars': {
                        'sorted_T2': {
                            '$sortArray': {
                                'input': '$T2_values',
                                'sortBy': 1
                            }
                        }
                    },
                    'in': {
                        '$arrayElemAt': [
                            '$$sorted_T2', {
                                '$floor': {
                                    '$divide': [
                                        {
                                            '$size': '$$sorted_T2'
                                        }, 2
                                    ]
                                }
                            }
                        ]
                    }
                }
            },
            'median_A31': {
                '$cond': {
                    'if': {
                        '$gt': [
                            {
                                '$size': {
                                    '$filter': {
                                        'input': '$A31_values',
                                        'as': 'value',
                                        'cond': {
                                            '$ne': [
                                                '$$value', None
                                            ]
                                        }
                                    }
                                }
                            }, 0
                        ]
                    },
                    'then': {
                        '$let': {
                            'vars': {
                                'sorted_A31': {
                                    '$sortArray': {
                                        'input': {
                                            '$filter': {
                                                'input': '$A31_values',
                                                'as': 'value',
                                                'cond': {
                                                    '$ne': [
                                                        '$$value', None
                                                    ]
                                                }
                                            }
                                        },
                                        'sortBy': 1
                                    }
                                }
                            },
                            'in': {
                                '$arrayElemAt': [
                                    '$$sorted_A31', {
                                        '$floor': {
                                            '$divide': [
                                                {
                                                    '$size': '$$sorted_A31'
                                                }, 2
                                            ]
                                        }
                                    }
                                ]
                            }
                        }
                    },
                    'else': None
                }
            },
            'median_T3': {
                '$let': {
                    'vars': {
                        'sorted_T3': {
                            '$sortArray': {
                                'input': '$T3_values',
                                'sortBy': 1
                            }
                        }
                    },
                    'in': {
                        '$arrayElemAt': [
                            '$$sorted_T3', {
                                '$floor': {
                                    '$divide': [
                                        {
                                            '$size': '$$sorted_T3'
                                        }, 2
                                    ]
                                }
                            }
                        ]
                    }
                }
            },
            'median_A32': {
                '$cond': {
                    'if': {
                        '$gt': [
                            {
                                '$size': {
                                    '$filter': {
                                        'input': '$A32_values',
                                        'as': 'value',
                                        'cond': {
                                            '$ne': [
                                                '$$value', None
                                            ]
                                        }
                                    }
                                }
                            }, 0
                        ]
                    },
                    'then': {
                        '$let': {
                            'vars': {
                                'sorted_A32': {
                                    '$sortArray': {
                                        'input': {
                                            '$filter': {
                                                'input': '$A32_values',
                                                'as': 'value',
                                                'cond': {
                                                    '$ne': [
                                                        '$$value', None
                                                    ]
                                                }
                                            }
                                        },
                                        'sortBy': 1
                                    }
                                }
                            },
                            'in': {
                                '$arrayElemAt': [
                                    '$$sorted_A32', {
                                        '$floor': {
                                            '$divide': [
                                                {
                                                    '$size': '$$sorted_A32'
                                                }, 2
                                            ]
                                        }
                                    }
                                ]
                            }
                        }
                    },
                    'else': None
                }
            },
            'median_T4': {
                '$let': {
                    'vars': {
                        'sorted_T4': {
                            '$sortArray': {
                                'input': '$T4_values',
                                'sortBy': 1
                            }
                        }
                    },
                    'in': {
                        '$arrayElemAt': [
                            '$$sorted_T4', {
                                '$floor': {
                                    '$divide': [
                                        {
                                            '$size': '$$sorted_T4'
                                        }, 2
                                    ]
                                }
                            }
                        ]
                    }
                }
            },
            'median_A33': {
                '$cond': {
                    'if': {
                        '$gt': [
                            {
                                '$size': {
                                    '$filter': {
                                        'input': '$A33_values',
                                        'as': 'value',
                                        'cond': {
                                            '$ne': [
                                                '$$value', None
                                            ]
                                        }
                                    }
                                }
                            }, 0
                        ]
                    },
                    'then': {
                        '$let': {
                            'vars': {
                                'sorted_A33': {
                                    '$sortArray': {
                                        'input': {
                                            '$filter': {
                                                'input': '$A33_values',
                                                'as': 'value',
                                                'cond': {
                                                    '$ne': [
                                                        '$$value', None
                                                    ]
                                                }
                                            }
                                        },
                                        'sortBy': 1
                                    }
                                }
                            },
                            'in': {
                                '$arrayElemAt': [
                                    '$$sorted_A33', {
                                        '$floor': {
                                            '$divide': [
                                                {
                                                    '$size': '$$sorted_A33'
                                                }, 2
                                            ]
                                        }
                                    }
                                ]
                            }
                        }
                    },
                    'else': None
                }
            },
            'median_T5': {
                '$let': {
                    'vars': {
                        'sorted_T5': {
                            '$sortArray': {
                                'input': '$T5_values',
                                'sortBy': 1
                            }
                        }
                    },
                    'in': {
                        '$arrayElemAt': [
                            '$$sorted_T5', {
                                '$floor': {
                                    '$divide': [
                                        {
                                            '$size': '$$sorted_T5'
                                        }, 2
                                    ]
                                }
                            }
                        ]
                    }
                }
            },
            'median_A34': {
                '$cond': {
                    'if': {
                        '$gt': [
                            {
                                '$size': {
                                    '$filter': {
                                        'input': '$A34_values',
                                        'as': 'value',
                                        'cond': {
                                            '$ne': [
                                                '$$value', None
                                            ]
                                        }
                                    }
                                }
                            }, 0
                        ]
                    },
                    'then': {
                        '$let': {
                            'vars': {
                                'sorted_A34': {
                                    '$sortArray': {
                                        'input': {
                                            '$filter': {
                                                'input': '$A34_values',
                                                'as': 'value',
                                                'cond': {
                                                    '$ne': [
                                                        '$$value', None
                                                    ]
                                                }
                                            }
                                        },
                                        'sortBy': 1
                                    }
                                }
                            },
                            'in': {
                                '$arrayElemAt': [
                                    '$$sorted_A34', {
                                        '$floor': {
                                            '$divide': [
                                                {
                                                    '$size': '$$sorted_A34'
                                                }, 2
                                            ]
                                        }
                                    }
                                ]
                            }
                        }
                    },
                    'else': None
                }
            },
            'median_T6': {
                '$let': {
                    'vars': {
                        'sorted_T6': {
                            '$sortArray': {
                                'input': '$T6_values',
                                'sortBy': 1
                            }
                        }
                    },
                    'in': {
                        '$arrayElemAt': [
                            '$$sorted_T6', {
                                '$floor': {
                                    '$divide': [
                                        {
                                            '$size': '$$sorted_T6'
                                        }, 2
                                    ]
                                }
                            }
                        ]
                    }
                }
            },
            'median_A35': {
                '$cond': {
                    'if': {
                        '$gt': [
                            {
                                '$size': {
                                    '$filter': {
                                        'input': '$A35_values',
                                        'as': 'value',
                                        'cond': {
                                            '$ne': [
                                                '$$value', None
                                            ]
                                        }
                                    }
                                }
                            }, 0
                        ]
                    },
                    'then': {
                        '$let': {
                            'vars': {
                                'sorted_A35': {
                                    '$sortArray': {
                                        'input': {
                                            '$filter': {
                                                'input': '$A35_values',
                                                'as': 'value',
                                                'cond': {
                                                    '$ne': [
                                                        '$$value', None
                                                    ]
                                                }
                                            }
                                        },
                                        'sortBy': 1
                                    }
                                }
                            },
                            'in': {
                                '$arrayElemAt': [
                                    '$$sorted_A35', {
                                        '$floor': {
                                            '$divide': [
                                                {
                                                    '$size': '$$sorted_A35'
                                                }, 2
                                            ]
                                        }
                                    }
                                ]
                            }
                        }
                    },
                    'else': None
                }
            }
        }
    }, {
        '$sort': {
            'hour': 1
        }
    }
]

In [18]:
import pandas as pd


# Convert the results to a pandas DataFrame
df = pd.DataFrame(list(collection.aggregate(combined_pipeline)))

In [19]:
df.columns

Index(['hour', 'median_A30', 'median_A31', 'median_A32', 'median_T4',
       'median_A33', 'median_T5', 'median_A34', 'median_T6', 'median_A35',
       'median_T1', 'median_T3', 'median_T2'],
      dtype='object')

In [20]:
df['diff_T1_A30'] = df['median_A30'] - df['median_T1']
df['diff_T2_A31'] = df['median_A31'] - df['median_T2']
df['diff_T3_A32'] = df['median_A32'] - df['median_T3']
df['diff_T4_A33'] = df['median_A33'] - df['median_T4']
df['diff_T5_A34'] = df['median_A34'] - df['median_T5']
df['diff_T6_A35'] = df['median_A35'] - df['median_T6']

In [23]:
df[['diff_T1_A30', 'diff_T2_A31', 'diff_T3_A32',
    "diff_T4_A33", "diff_T5_A34", "diff_T6_A35"]].describe()

,diff_T1_A30,diff_T2_A31,diff_T3_A32,diff_T4_A33,diff_T5_A34,diff_T6_A35
count,425.000000,521.000000,552.000000,587.000000,378.000000,526.000000
mean,2.351324,0.619842,3.078012,2.335285,2.700231,3.738356
std,0.405796,0.413430,0.656149,0.194171,0.816750,0.142045
min,1.375000,-0.062500,1.187500,1.750000,-1.500000,2.375000
25%,2.062500,0.312500,2.750000,2.187500,2.515625,3.687500
50%,2.312500,0.500000,3.187500,2.312500,2.750000,3.750000
75%,2.625000,0.937500,3.500000,2.437500,2.937500,3.812500
max,3.687500,1.687500,4.937500,2.750000,5.187500,4.000000


In [24]:
import plotly.graph_objs as go

def plot_median_series(df, series1, series2, title='Temperatures of TLeaf and ALeaf'):
    fig = go.Figure()

    # Add the first series
    fig.add_trace(go.Scatter(
        x=df['hour'], y=df[series1], mode='lines', name=f'Median {series1.split("_")[1]}'))

    # Add the second series
    fig.add_trace(go.Scatter(
        x=df['hour'], y=df[series2], mode='lines', name=f'Median {series2.split("_")[1]}'))

    # Update layout
    fig.update_layout(title=title,
                      xaxis_title='Hour',
                      yaxis_title='Median Values',
                      legend_title='Series',
                      legend=dict(
                          orientation="h",
                          yanchor="bottom",
                          y=1.02,
                          xanchor="left",
                          x=0.5
                      ),
                      hovermode="x unified"
                      )

    # Show the figure
    fig.show()

In [12]:
# Example usage
plot_median_series(df, 'median_T1', 'median_A30')

In [13]:
# Example usage
plot_median_series(df, 'median_T2', 'median_A31')

In [14]:
# Example usage
plot_median_series(df, 'median_T3', 'median_A32')

In [25]:
# Example usage
plot_median_series(df, 'median_T4', 'median_A33')

In [26]:
# Example usage
plot_median_series(df, 'median_T5', 'median_A34')

In [27]:
# Example usage
plot_median_series(df, 'median_T6', 'median_A35')

In [15]:
import plotly.graph_objects as go

# Create traces for the differences
trace_diff_1 = go.Scatter(
    x=df['hour'],
    y=df['diff_T1_A30'],
    mode='lines',
    name='Diff T1-A30'
)

trace_diff_2 = go.Scatter(
    x=df['hour'],
    y=df['diff_T2_A31'],
    mode='lines',
    name='Diff T2-A31'
)

trace_diff_3 = go.Scatter(
    x=df['hour'],
    y=df['diff_T3_A32'],
    mode='lines',
    name='Diff T3-A32'
)

# Create a figure and add traces
fig = go.Figure()

fig.add_trace(trace_diff_1)
fig.add_trace(trace_diff_2)
fig.add_trace(trace_diff_3)

# Update the layout with titles and legend positioning
fig.update_layout(
    title='Temperature Differences Between TLeaf and ALeaf Over Time',
    xaxis_title='Hour',
    yaxis_title='Difference',
    legend_title='Series',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    hovermode="x unified"
)

# Show the figure
fig.show()